In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# ターゲットURL
url = "https://suumo.jp/chintai/tokyo/sa_other_02/mansion/"

# ページ数（最大5ページ）
max_pages = 5

# 物件情報を格納するリスト
property_info_list = []

# 1秒待機
time.sleep(1)

# ページごとにスクレイピング

for page in range(1, max_pages + 1):
    page_url = f"{url}?page={page}"
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # 物件情報を取得
    for item in soup.find_all(class_="cassetteitem"):
        property_info = {
            "物件名": item.find(class_="cassetteitem_content-title").text.strip(),
            "所在地": item.find(class_="cassetteitem_detail-col1").text.strip(),
            "交通": item.find(class_="cassetteitem_detail-col2").text.strip(),
            "築年数": item.find(class_="cassetteitem_detail-col3").text.strip(),
        }
        # 階、賃料、管理費、間取り、専有面積を別の行に追加
        for row in item.find_all(class_="js-cassette_link"):
            floor = row.find_all("td")[2].text.strip().replace("階", "")
            rent = row.find(class_="cassetteitem_price--rent").text.strip().replace("万円", "").replace("-", "0")
            admin_fee = row.find(class_="cassetteitem_price--administration").text.strip().replace("円", "").replace("万円", "").replace("-", "0")
            layout = row.find(class_="cassetteitem_madori").text.strip()
            area = row.find(class_="cassetteitem_menseki").text.strip().replace("m2", "")
            property_info.update({"階": floor, "賃料": float(rent) * 10000, "管理費": float(admin_fee), "間取り": layout, "専有面積": area})
            property_info_list.append(property_info.copy())  # コピーを追加

# データフレームに変換
df = pd.DataFrame(property_info_list)

# 修正後の DataFrame を表示
df


,物件名,所在地,交通,築年数,階,賃料,管理費,間取り,専有面積
0,ライフピア桜,東京都足立区千住桜木１,ＪＲ常磐線/北千住駅 歩12分\n京成本線/千住大橋駅 歩13分\n都電荒川線/荒川七丁目駅...,築91年\n7階建,6,65000.0,4500.0,1DK,23.04
1,都営新宿線 一之江駅 4階建 築59年,東京都江戸川区松江２,都営新宿線/一之江駅 歩25分\n都営新宿線/船堀駅 歩26分\n都営新宿線/瑞江駅 歩32分,築59年\n4階建,3,63000.0,0.0,2DK,27.89
2,東京メトロ日比谷線 三ノ輪駅 3階建 築80年,東京都荒川区南千住２,東京メトロ日比谷線/三ノ輪駅 歩5分\n東京メトロ日比谷線/南千住駅 歩7分\nＪＲ常磐線/...,築80年\n3階建,3,90000.0,3000.0,2DK,39.52
3,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年\n7階建,4,56500.0,3000.0,ワンルーム,24.6
4,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年\n7階建,2,60000.0,3000.0,ワンルーム,23.26
...,...,...,...,...,...,...,...,...,...
161,渕江マンション,東京都足立区東保木間２,つくばエクスプレス/六町駅 歩20分\nつくばエクスプレス/青井駅 歩38分\n東武伊勢崎線...,築52年\n3階建,2,63000.0,2000.0,1LDK,35
162,七澤ビル,東京都台東区浅草５,東京メトロ銀座線/浅草駅 歩15分\nつくばエクスプレス/浅草駅 歩13分,築57年\n3階建,3,65000.0,0.0,1DK,24.79
163,菅原ハイツ,東京都江東区東砂３,都営新宿線/東大島駅 歩20分\n東京メトロ東西線/南砂町駅 歩22分\n都営新宿線/大島駅...,築52年\n3階建,1,65000.0,0.0,1K,22
164,東京メトロ東西線 南砂町駅 3階建 築40年,東京都江東区東砂３,東京メトロ東西線/南砂町駅 歩21分\n都営新宿線/東大島駅 歩20分\nＪＲ総武線/亀戸駅...,築40年\n3階建,1,65000.0,0.0,ワンルーム,22


In [4]:
import re

# 築年数の2行目を削除
df["築年数"] = df["築年数"].str.split("\n").str[0]

# 所在地から「東京都」の後から「○○区」までを抜き出す関数
def extract_district(location):
    match = re.search(r'東京都(.+?区)', location)
    if match:
        return match.group(1)
    else:
        return None

# 区という項目に出力
df['区'] = df['所在地'].apply(extract_district)

# 修正後の DataFrame を表示
df


,物件名,所在地,交通,築年数,階,賃料,管理費,間取り,専有面積,区
0,ライフピア桜,東京都足立区千住桜木１,ＪＲ常磐線/北千住駅 歩12分\n京成本線/千住大橋駅 歩13分\n都電荒川線/荒川七丁目駅...,築91年,6,65000.0,4500.0,1DK,23.04,足立区
1,都営新宿線 一之江駅 4階建 築59年,東京都江戸川区松江２,都営新宿線/一之江駅 歩25分\n都営新宿線/船堀駅 歩26分\n都営新宿線/瑞江駅 歩32分,築59年,3,63000.0,0.0,2DK,27.89,江戸川区
2,東京メトロ日比谷線 三ノ輪駅 3階建 築80年,東京都荒川区南千住２,東京メトロ日比谷線/三ノ輪駅 歩5分\n東京メトロ日比谷線/南千住駅 歩7分\nＪＲ常磐線/...,築80年,3,90000.0,3000.0,2DK,39.52,荒川区
3,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年,4,56500.0,3000.0,ワンルーム,24.6,江戸川区
4,三家第1ビル,東京都江戸川区中央３,ＪＲ総武線快速/新小岩駅 歩20分\nＪＲ総武線/小岩駅 歩30分\n都営新宿線/船堀駅 バ...,築61年,2,60000.0,3000.0,ワンルーム,23.26,江戸川区
...,...,...,...,...,...,...,...,...,...,...
161,渕江マンション,東京都足立区東保木間２,つくばエクスプレス/六町駅 歩20分\nつくばエクスプレス/青井駅 歩38分\n東武伊勢崎線...,築52年,2,63000.0,2000.0,1LDK,35,足立区
162,七澤ビル,東京都台東区浅草５,東京メトロ銀座線/浅草駅 歩15分\nつくばエクスプレス/浅草駅 歩13分,築57年,3,65000.0,0.0,1DK,24.79,台東区
163,菅原ハイツ,東京都江東区東砂３,都営新宿線/東大島駅 歩20分\n東京メトロ東西線/南砂町駅 歩22分\n都営新宿線/大島駅...,築52年,1,65000.0,0.0,1K,22,江東区
164,東京メトロ東西線 南砂町駅 3階建 築40年,東京都江東区東砂３,東京メトロ東西線/南砂町駅 歩21分\n都営新宿線/東大島駅 歩20分\nＪＲ総武線/亀戸駅...,築40年,1,65000.0,0.0,ワンルーム,22,江東区


In [9]:
#import gspread
#from gspread_dataframe import set_with_dataframe

# Google Sheets の認証
#gc = gspread.service_account(filename='spread-sheet-api-419202-59ee0bee5075.json')  # 認証情報ファイルのパスを指定

# スプレッドシートを開く（スプレッドシートの URL から ID を取得して指定）
#spreadsheet_id = '1S6f9kJ7in52I8d7OxEQrZQCx80KgYoGmCJhVhloen0Q'
#worksheet_name = '賃貸物件'
#sh = gc.open_by_key(spreadsheet_id)
#worksheet = sh.worksheet(worksheet_name)

# DataFrame をスプレッドシートに書き込む
#set_with_dataframe(worksheet, df)


In [7]:
import sqlite3

# SQLite データベースに接続
conn = sqlite3.connect('chintai_database.db')
cursor = conn.cursor()

# テーブル作成
cursor.execute('CREATE TABLE IF NOT EXISTS my_table (id INTEGER PRIMARY KEY, 物件名 TEXT, 所在地 TEXT, 交通 TEXT, 築年数 TEXT, 階 INTEGER, 賃料 REAL, 管理費 REAL, 間取り TEXT, 専有面積 REAL, 区 TEXT,latitude REAL,longitude REAL)')
# データ挿入
for _, row in df.iterrows():
    cursor.execute('INSERT INTO my_table (物件名, 所在地, 交通, 築年数, 階, 賃料, 管理費, 間取り, 専有面積, 区) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', 
                   (row['物件名'], row['所在地'], row['交通'], row['築年数'], row['階'], row['賃料'], row['管理費'], row['間取り'], row['専有面積'], row['区']))

# コミットと接続のクローズ
conn.commit()
conn.close()


In [6]:
# データの取り出し例
def get_data_from_db():
    conn = sqlite3.connect('chintai_database.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM my_table WHERE 賃料 > ?', (100000,))
    results = cursor.fetchall()
    conn.close()
    return results

# データの取り出し
results = get_data_from_db()
for row in results:
    print(row)

(1, '近藤ビル', '東京都台東区根岸５', '東京メトロ日比谷線/三ノ輪駅 歩7分\nＪＲ山手線/鶯谷駅 歩16分\n東京メトロ日比谷線/入谷駅 歩12分', '築42年', 3, 115000.0, 0.0, '1LDK', 54.02, '台東区')
(7, 'ＪＲ山手線 西日暮里駅 4階建 築68年', '東京都荒川区西日暮里１', 'ＪＲ山手線/西日暮里駅 歩12分\nＪＲ山手線/日暮里駅 歩13分\nＪＲ常磐線/三河島駅 歩2分', '築68年', 4, 120000.0, 5000.0, '2SLDK', 59.96, '荒川区')
(11, 'マ・セゾン', '東京都荒川区西日暮里１', 'ＪＲ山手線/西日暮里駅 歩12分\nＪＲ山手線/日暮里駅 歩13分\nＪＲ常磐線/三河島駅 歩2分', '築68年', 4, 120000.0, 5000.0, '2SLDK', 59.96, '荒川区')
(44, 'なぎさニュータウン５号棟', '東京都江戸川区南葛西７', '東京メトロ東西線/葛西駅 歩20分', '築48年', 14, 135000.0, 0.0, '3LDK', 71.5, '江戸川区')
(45, 'なぎさニュータウン５号棟', '東京都江戸川区南葛西７', '東京メトロ東西線/葛西駅 歩20分', '築48年', 14, 135000.0, 0.0, '3LDK', 71.5, '江戸川区')
(46, 'コスモ新小岩プレミール', '東京都江戸川区松江２', '都営新宿線/船堀駅 バス18分 (バス停)大杉東小学校前 歩6分\nＪＲ総武線/新小岩駅 バス18分 (バス停)中央通り 歩4分\n東京メトロ東西線/葛西駅 バス25分 (バス停)中央通り 歩3分', '築36年', 3, 178000.0, 12000.0, '4LDK', 102.36, '江戸川区')
(47, 'コスモ新小岩プレミール', '東京都江戸川区松江２', '都営新宿線/船堀駅 バス18分 (バス停)大杉東小学校前 歩6分\nＪＲ総武線/新小岩駅 バス18分 (バス停)中央通り 歩4分\n東京メトロ東西線/葛西駅 バス25分 (バス停)中央通り 歩3分', '築36年', 2, 220000.0, 0.0, '4LDK', 102.36, '江戸川区')